In [0]:
CREATE OR REPLACE TEMPORARY FUNCTION clean_date(d DATE)
RETURNS DATE
RETURN CASE
  WHEN d IS NULL THEN NULL
  WHEN d < DATE '1901-01-01' THEN NULL
  WHEN d IN (
    DATE '1900-01-01', DATE '1901-01-01',
    DATE '1969-12-31', DATE '1970-01-01',
    DATE '2099-12-31', DATE '2999-12-31', DATE '9999-12-31'
  ) THEN NULL
  WHEN d > current_date() + INTERVAL 3 YEARS THEN NULL
  ELSE d
END;

--CREATE OR REPLACE TABLE teamconnect.pca_prism.02_tbl_account_prism_services AS
INSERT OVERWRITE TABLE pca_prism.02_tbl_account_prism_services
WITH csactar_btn_map AS (
  SELECT DISTINCT
    CASE WHEN LENGTH(REGEXP_REPLACE(CAST(actexc AS STRING),'[^0-9]',''))=6
           AND LENGTH(REGEXP_REPLACE(CAST(actlne AS STRING),'[^0-9]','')) BETWEEN 1 AND 4
         THEN CONCAT(
           REGEXP_REPLACE(CAST(actexc AS STRING),'[^0-9]',''),
           LPAD(REGEXP_REPLACE(CAST(actlne AS STRING),'[^0-9]',''),4,'0')
         )
    END AS btn,
    UPPER(TRIM(env)) AS dpi_environment,
    CAST(srvid AS BIGINT) AS srvid
  FROM it_bronze_prod.dpi.csactar
  WHERE srvid IS NOT NULL AND actexc IS NOT NULL AND actlne IS NOT NULL
),
svc_raw AS (
  SELECT
    UPPER(TRIM(env)) AS dpi_environment,
    CAST(srvid AS BIGINT) AS srvid,
    TRIM(ftrint) AS feature_code,
    TRIM(ftrdes) AS feature_desc,
    TRIM(pkgdes) AS package_desc,
    TRIM(prdcls) AS product_class,
    TRIM(ftrctg) AS feature_category,
    TRIM(ftrgrp) AS feature_group,
    UPPER(TRIM(ftrtyp)) AS feature_type,
    CAST(chrrte AS DECIMAL(15,5)) AS rate,
    CAST(chrqty AS DECIMAL(10,0)) AS quantity,
    CAST(chramt AS DECIMAL(15,5)) AS amount,
    clean_date(effdte) AS eff_date,
    clean_date(cnldte) AS cancel_date,
    clean_date(lbldte) AS last_billed_date
  FROM it_gold_prod.dpi.csftrv1
  WHERE srvid IS NOT NULL
),
svc_with_btn AS (
  SELECT
    m.btn, r.dpi_environment, r.srvid,
    r.feature_code, r.feature_desc, r.package_desc,
    r.product_class, r.feature_category, r.feature_group, r.feature_type,
    r.rate, r.quantity, r.amount,
    r.eff_date, r.cancel_date, r.last_billed_date,
    COALESCE(r.last_billed_date, r.eff_date) AS billed_date
  FROM svc_raw r
  LEFT JOIN csactar_btn_map m
    ON m.srvid = r.srvid AND m.dpi_environment = r.dpi_environment
  WHERE m.btn IS NOT NULL AND m.btn RLIKE '^[0-9]{10}$'
),
svc_current_active AS (
  SELECT
    btn, dpi_environment,
    COALESCE(SUM(amount), 0) AS current_mrc_sum,
    COLLECT_LIST(named_struct(
      'feature_code',feature_code,'feature_desc',feature_desc,'package_desc',package_desc,
      'product_class',product_class,'feature_category',feature_category,'feature_group',feature_group,
      'feature_type',feature_type,'rate',rate,'quantity',quantity,'amount',amount,
      'effective_date',eff_date,'cancel_date',cancel_date,'billed_date',billed_date
    )) AS current_services
  FROM svc_with_btn
  WHERE (cancel_date IS NULL OR cancel_date > current_date())
    AND (eff_date IS NULL OR eff_date <= current_date())
  GROUP BY 1,2
),
svc_current_month AS (
  SELECT
    btn, dpi_environment,
    COALESCE(SUM(amount), 0) AS current_month_mrc_sum,
    COLLECT_LIST(named_struct(
      'billed_date', billed_date, 'feature_code', feature_code,
      'feature_desc', feature_desc, 'amount', amount
    )) AS current_month_itemized_services
  FROM svc_with_btn
  WHERE billed_date >= DATE_TRUNC('month', current_date())
    AND billed_date <  ADD_MONTHS(DATE_TRUNC('month', current_date()), 1)
  GROUP BY 1,2
)
SELECT
  b.btn,
  b.dpi_environment,
  COALESCE(svcA.current_mrc_sum, 0)                        AS current_mrc_sum,
  COALESCE(svcA.current_services, ARRAY())                 AS current_services,
  COALESCE(svcCM.current_month_mrc_sum, 0)                 AS current_month_mrc_sum,
  COALESCE(svcCM.current_month_itemized_services, ARRAY()) AS current_month_itemized_services
FROM (
  SELECT DISTINCT btn, dpi_environment
  FROM svc_with_btn
) b
LEFT JOIN svc_current_active svcA ON svcA.btn = b.btn AND svcA.dpi_environment = b.dpi_environment
LEFT JOIN svc_current_month svcCM ON svcCM.btn = b.btn AND svcCM.dpi_environment = b.dpi_environment;

OPTIMIZE teamconnect.pca_prism.02_tbl_account_prism_services
ZORDER BY (btn, dpi_environment);
